# Valideren van alle karteringen

Deze notebook valideert door veg2hab omgezette karteringen adhv 'officiële' handmatig omgezette habitatkarteringen en geeft een overzicht van de resultaten.

**KIJK UIT** Deze notebook zet de karteringen niet om, dat moet los met de `alle_karteringen_door_veg2hab.ipynb` notebook. 


In [1]:
import geopandas as gpd
import pandas as pd

from veg2hab import validation

import geopandas as gpd

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

omgezette_karteringen_prefix = "../testing/omgezette_vegetatiekarteringen/"
habitatkarteringen_prefix = "../testing/"

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Inladen van de predictions en de trues
We leggen ook meteen de percentages toegekende kwaliteit vast, als we de preds toch ingeladen hebben

In [2]:
validationinfo = pd.read_excel(
    "../testing/validationinfo.xlsx", 
    engine='openpyxl', 
    usecols=[
        "path_pred",
        "naam_in_overzicht_habkart",
        "provincie",
        "path_true",
        "habtype_cols_regex",
        "percentage_cols_regex",
        "layer",
        "werkt_nu",
    ]
)

# Omzetten van pd.NA naar None om te voorkomen dat we overal "if not pd.isna()" moeten doen
validationinfo = validationinfo.applymap(lambda x: None if pd.isna(x) else x)

# Uitfilteren van karteringen die we om wat voor reden dan ook niet goed om kunnen zetten (zie kolom "reden dat hij niet werkt" in de excel)
validationinfo = validationinfo[validationinfo.werkt_nu]

validationinfo["path_true"] = validationinfo["path_true"].apply(lambda x: "../testing/" + x[2:])
validationinfo["path_pred"] = validationinfo["path_pred"].apply(lambda x: omgezette_karteringen_prefix + x[2:])
validationinfo

# Load in the habitatkarteringen
trues = {}
preds = {}
perc_kwals = {}
for row in validationinfo.itertuples():
    # We printen even wat dingen zodat je kunt zien waar errors vandaan komen (als die er zijn)
    print(f"----- {row.naam_in_overzicht_habkart}")
    print("--- trues")
    # Habitatkarteringen (trues)
    raw = gpd.read_file(row.path_true, layer=row.layer)
    parsed = validation.parse_habitat_percentages(
        raw, 
        habtype_cols_regex=row.habtype_cols_regex, 
        percentage_cols_regex=row.percentage_cols_regex, 
        how_to_handle_missing_percentages="split_equally" if row.percentage_cols_regex is None else None)
    trues[row.naam_in_overzicht_habkart] = parsed
    print("--- preds")
    # Door ons omgezette vegetatiekarteringen (preds)
    raw = gpd.read_file(row.path_pred)
    parsed = validation.parse_habitat_percentages(raw, add_kwaliteit=True)
    preds[row.naam_in_overzicht_habkart] = parsed

----- SBB Westerwolde 2020
--- trues
--- preds
----- SBB ZWK 2010 0814_Tussen de Gasten 2010
--- trues
--- preds
----- SBB ZWK 2010 0815_Trimunt_2010
--- trues
--- preds
----- GL_Zuidlaardermeer2019
--- trues
--- preds
----- SGL Hunzedal en Leekstermeer2021 Leekstermeer 2021
--- trues


--- preds
----- SBB Lauwersmeer 2015
--- trues
--- preds
----- SGL Hunzedal en Leekstermeer2021 Hunzedal
--- trues


--- preds


----- Schiermonnikoog_2015
--- trues
--- preds
----- Vlieland_2013
--- trues
--- preds
----- NM_Mantingerbos_2015
--- trues
--- preds
----- SBB1034_Dwingelderveld2017
--- trues
--- preds
----- Drouwenerzand_2020
--- trues
--- preds
----- NM_Fochteloerveen2013_14
--- trues
--- preds
----- NM_SBB_Fochteloerveen_randen2016
--- trues
--- preds
----- SBB0816_Norg2010
--- trues
--- preds
----- NM_Fochteloerveen_2020
--- trues
--- preds
----- BosGroep_SNLMonitoring_2014
--- trues
--- preds
----- NM_DCR_2013
--- trues
--- preds
----- PRV_Witterdiep2021
--- trues
--- preds


### Combineren van de preds en de trues

Bij het over elkaar heen leggen van de vlakken in preds en die in trues ontstaan er soms heel veel kleine slivervlakken; deze worden verwijderd door alle vlakken kleiner dan 1m² te droppen. Of en hoeveel vlakken er gedropt worden wordt hier beneden gerapporteerd. Als hier grote getallen uitkomen (ruwweg 1 m² per 1000 rows dropped is normaal) is er wellicht wat mis gegaan.

In [3]:
for key in trues.keys():
    assert key in preds.keys(), f"Key {key} not found in predictions"

output = {}
for key, true in trues.items():
    print(f"---- {key}")
    pred = preds[key]
    combined = validation.spatial_join(pred, true, how="intersection")
    output[key] = validation.voeg_correctheid_toe_aan_df(combined)

---- SBB Westerwolde 2020


---- SBB ZWK 2010 0814_Tussen de Gasten 2010


---- SBB ZWK 2010 0815_Trimunt_2010
---- GL_Zuidlaardermeer2019


---- SGL Hunzedal en Leekstermeer2021 Leekstermeer 2021


---- SBB Lauwersmeer 2015


...

---- NM_SBB_Fochteloerveen_randen2016


---- SBB0816_Norg2010


---- NM_Fochteloerveen_2020


---- BosGroep_SNLMonitoring_2014
---- NM_DCR_2013


---- PRV_Witterdiep2021


### OUTPUTS

In [4]:
def format_dict(dict):
    string = ""
    for key, value in dict.items():
        string += f"{key}: {value:.2f}\n"
    return string[:-1] # Remove last newline

Percentage/vlakken

In [5]:
for key, gdf in output.items():
    print("------ " + key)
    print("--- CONFUSION MATRIX:")
    conf_matrix = validation.bereken_volledige_conf_matrix(gdf, method="percentage")
    print(conf_matrix)
    print("--- F1 METRICS")
    print(format_dict(validation.bereken_F1_per_habtype(conf_matrix, method="percentage")))
    print(f"Gemiddelde F1: {validation.bereken_gemiddelde_F1(conf_matrix, method='percentage'):.3f}")
    print("--- TOEGEKENDE HABTYPEN:")
    toegekende_habtypen = validation.bereken_totaal_from_dict_col(gdf, "pred_hab_perc", method="percentage")
    print(format_dict(toegekende_habtypen))
    sum_habtypes = sum(toegekende_habtypen.values())
    print(f"Percentage niet-HXXXX: {((sum_habtypes - toegekende_habtypen.get('HXXXX', 0)) / sum_habtypes * 100):.3f}%")
    print("--- TOEGEKENDE KWALITEIT:")
    uncombined_gdf = preds[key]
    print(format_dict(validation.bereken_totaal_from_dict_col(uncombined_gdf, "kwal_perc", method="percentage")))
    print("------------------------")

------ SBB Westerwolde 2020
--- CONFUSION MATRIX:
true_hab   H0000  H3130  H3160  H4010A  H4030  H6230  H6410  H6430A  H7110B  H7150  H91D0  HXXXX
pred_hab                                                                                        
H0000     689.27    0.0   2.68    1.23   2.39  11.26    0.0     0.2     0.0    0.0   0.49    0.0
H3130       0.00    0.0   0.00    0.00   0.00   0.00    0.0     0.0     0.0    0.0   0.00    0.0
H3160       0.00    0.0   0.00    0.00   0.00   0.00    0.0     0.0     0.0    0.0   0.00    0.0
H4010A      0.50    0.0   0.30    7.53   0.30   0.00    0.0     0.0     0.0    0.0   0.00    0.0
H4030       0.00    0.0   0.00    0.00   0.00   0.00    0.0     0.0     0.0    0.0   0.00    0.0
H6230       0.00    0.0   0.00    0.00   0.00   0.00    0.0     0.0     0.0    0.0   0.00    0.0
H6410       0.00    0.0   0.00    0.00   0.00   0.00    0.0     0.0     0.0    0.0   0.00    0.0
H6430A      0.00    0.0   0.00    0.00   0.00   0.00    0.0     0.0     0.0  

Area/oppervlak

In [6]:
for key, gdf in output.items():
    print("------ " + key)
    print("--- CONFUSION MATRIX:")
    conf_matrix = validation.bereken_volledige_conf_matrix(gdf, method="area")
    print(conf_matrix)
    print("--- F1 METRICS")
    print(format_dict(validation.bereken_F1_per_habtype(conf_matrix, method="area")))
    print(f"Gemiddelde F1: {validation.bereken_gemiddelde_F1(conf_matrix, method='area'):.3f}")
    print("--- TOEGEKENDE HABTYPEN:")
    toegekende_habtypen = validation.bereken_totaal_from_dict_col(gdf, "pred_hab_perc", method="area")
    print(format_dict(toegekende_habtypen))
    sum_habtypes = sum(toegekende_habtypen.values())
    print(f"Percentage niet-HXXXX: {((sum_habtypes - toegekende_habtypen.get('HXXXX', 0)) / sum_habtypes * 100):.3f}%")
    print("--- TOEGEKENDE KWALITEIT:")
    uncombined_gdf = preds[key]
    print(format_dict(validation.bereken_totaal_from_dict_col(uncombined_gdf, "kwal_perc", method="area")))
    print("------------------------")

------ SBB Westerwolde 2020
--- CONFUSION MATRIX:
true_hab       H0000     H3130     H3160    H4010A      H4030     H6230     H6410    H6430A    H7110B     H7150     H91D0  HXXXX
pred_hab                                                                                                                        
H0000     242.753795  0.000000  0.233985  0.032755   0.470523  5.034103  0.000000  0.049636  0.000000  0.000000  0.079943    0.0
H3130       0.000000  0.000000  0.000000  0.000000   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000    0.0
H3160       0.000000  0.000000  0.000000  0.000000   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000    0.0
H4010A      0.233636  0.000000  0.092511  1.795340   0.084422  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000    0.0
H4030       0.000000  0.000000  0.000000  0.000000   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000    0.0
H6230       0.000000  0.000000  0.000000  0.000

Saving as gpkg

In [7]:
for key, gdf in output.items():
    gdf.to_file(f"../testing/validation_output/{key}.gpkg")